In [2]:
from datetime import datetime
from threading import Thread
from time import sleep
import numpy as np

In [3]:
def threaded(fn):
    def wrapper(*args, **kwargs):
        thread = Thread(target=fn, args=args, kwargs=kwargs)
        thread.start()
        return thread
    return wrapper

In [4]:
class Stepper:
    def __init__(self, id_, speedRPM = 60, direction = False):
        # Maximum allowed number of motors are 8 therefor id should between 0->7(inclusive)
        self.id = id_
        self.dir_pin = id_ # the direction pin
        self.step_pin = id_ + 1 # stepping pin
        
        self.speedRPM = speedRPM
        self.duration = 1/((self.speedRPM / 60) * 200) * 1e6  # period of a pulse in microseconds
        self.direction = direction

In [5]:
class StepperDriver:
    def __init__(self, motorRef):
        """
        DOCSTRING: This function will initialize the Stepper Driver Unit
        """
        self.conn = {
            'clk': 11, # Serial clock pin
            'ser': 10, # Serial data pin
            'ltc': 22  # Latch pin
        }
        
        self.motorRef = motorRef # a list of motor objects

        # now we have to fill in the empty motors
        for tempID in range(8):
            if tempID not in [motor.id for motor in self.motorRef]:
                self.motorRef.append(Stepper(tempID))
                
        
        self.pulseSpeed = 2670 #minimum requirenment
        self.pulsePeriod = 1/self.pulseSpeed * 1e6

        # extract motorDirections
        self.__motorDirections = np.array([motor.direction for motor in motorRef], dtype='int8')
        # calculate bit flip periods
        self.flipPeriods = self.__calculateBitPeriod()
        
        self.__setupGPIO()
        self.__exit = False

    def __setupGPIO(self):
        pass
#         GPIO.setmode(GPIO.BCM)
#         GPIO.setup(self.conn['clk'], GPIO.OUT)
#         GPIO.setup(self.conn['ser'], GPIO.OUT)
#         GPIO.setup(self.conn['ltc'], GPIO.OUT)

#         GPIO.output(self.conn['ltc'], GPIO.LOW)
#         GPIO.output(self.conn['clk'], GPIO.HIGH)

    def __shiftOut(self, val):
        sleep(70 * 1e-6)
#         for i in range(0,16):
#             GPIO.output(self.conn['clk'], GPIO.HIGH)
#             GPIO.output(self.conn['ser'], val[i] and GPIO.HIGH or GPIO.LOW)
#             GPIO.output(self.conn['clk'], GPIO.LOW)
        
    def __calculateBitPeriod(self):
        """
        DOCTRING: This function assumes the paralel data output at pulseSpeed and calculate the lipping points
                  each motor
        """
        flips = np.ones(shape = 8, dtype = 'int8')
        
        for motor in self.motorRef:
            flips[motor.id] = round(self.pulseSpeed / (400 * motor.speedRPM / 60))
            
        return flips
            

    @threaded
    def stepMotors(self):
        """
        DOCSTRING: Since we must approch 6.667 RPS from a motor, Pulsing Rate -> 6.667 * 200 -> 1333.4 Hz but pulse 
                   includes 1,0 Combination therefor step data must be generated in 2670 Hz minimum.
        """
        
        # FORMAT-> M0 M1 M2 M3 M4 M5 ..
        #       -> DS Ds Ds Ds Ds Ds ..
        
        __counter = 0 # The Flip Counter
        data = np.zeros(shape = (16), dtype=bool)
        
        while not self.__exit:
            start = datetime.now()
            
            data[::2] = self.__motorDirections
            data[1::2] ^= __counter % self.flipPeriods == 0
            

#             GPIO.output(self.conn['ltc'], GPIO.HIGH)
            self.__shiftOut(data[::-1])
#             GPIO.output(self.conn['ltc'], GPIO.LOW)

            try:
                sleep((self.pulsePeriod - (datetime.now() - start).microseconds - 100) / 1e6)
            except ValueError:
                pass

            __counter += 1 # increment the coounter
            print(data.astype('int8'))
            if __counter > 30: break

#         GPIO.cleanup()

    def __printMotors(self):
        print("+-------------+-------------+-------------+")
        print("|  MOTOR ID   |  SPEED RPM  |  DIRECTION  |")
        print("+-------------+-------------+-------------+")
        
        for motor in self.motorRef:
            print("|" + str(motor.id).ljust(13) + "|" + str(motor.speedRPM).ljust(13) + "|" + str(motor.direction).ljust(13) + "|")
        
        print("+-------------+-------------+-------------+")

    def __setAttr(self, id_, speed=None):
        if speed:
            self.motorRef[id_].speedRPM = speed
            self.flipPeriods = self.__calculateBitPeriod()
            

    @threaded
    def driverConsole(self):
        while not self.__exit:
            data = input("Driver Console#/ > ").split()
            
            if data[0] == 'quit()':
                self.__exit = True

            elif data[0] == 'list':
                if data[1] == 'motors':
                    self.__printMotors()

            elif data[0] == 'set':
                motorid = int(data[1][-1])
                
                if data[2] == 'speed':                    
                    self.motorRef[motorid].speedRPM = int(data[3])
                    self.flipPeriods = self.__calculateBitPeriod()
                
                elif data[2] == 'dir':
                    self.motorRef[motorid].direction = bool(int(data[3]))
                    self.__motorDirections = np.array([motor.direction for motor in motorRef], dtype='int8')

            

    def start(self):
        self.stepMotors()
        self.driverConsole()

In [6]:
driver = StepperDriver([Stepper(0), Stepper(1, speedRPM = 250, direction=True), Stepper(2, speedRPM=400)])
driver.start()

[0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 1 1 0 0 0 1 0 1 0 1 0 1 0 1]
[0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 1 0 0 0 0 1 0 1 0 1 0 1 0 1]
[0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 1 1 0 0 0 1 0 1 0 1 0 1 0 1]
[0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 1 0 0 0 0 1 0 1 0 1 0 1 0 1]
[0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 1 1 0 0 0 1 0 1 0 1 0 1 0 1]
[0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 1 0 0 0 0 1 0 1 0 1 0 1 0 1]
[0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 1 1 0 0 0

In [20]:
lista = [Stepper(0, speedRPM=120, direction=True),
         Stepper(1, speedRPM=350, direction=False),
         Stepper(2, speedRPM=46, direction=True),
         Stepper(3, speedRPM=12, direction=False),
         Stepper(4, speedRPM=20, direction=False),
         Stepper(5, speedRPM=56, direction=True),
         Stepper(6, speedRPM=230, direction=False),
         Stepper(7, speedRPM=200, direction=False)]

In [52]:
data_ = np.zeros(shape=16, dtype='int8')

In [53]:
motornp = np.array(lista)

In [54]:
expnp = np.array([1,2,3,4,5,6,7,8])

In [55]:
expnp[1::2]

array([2, 4, 6, 8])

In [56]:
# setting data directoins
__directions = np.array([motor.direction for motor in lista])
data_[::2] = __directions

print(data_)

[1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0]


In [73]:
flipsexp = np.array([7,3,4,1,1,1,1,1], dtype='int8')
counter_ = 0

In [77]:
for counter_ in range(20):
    
    data_[1::2] ^= counter_ % flipsexp == 0
    print(data_)

# counter_ % flipsexp == 0

[1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1]
[1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0]
[1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1]
[1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0]
[1 1 0 0 1 0 0 1 0 1 1 1 0 1 0 1]
[1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
[1 1 0 1 1 0 0 1 0 1 1 1 0 1 0 1]
[1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0]
[1 0 0 1 1 1 0 1 0 1 1 1 0 1 0 1]
[1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0]
[1 0 0 0 1 1 0 1 0 1 1 1 0 1 0 1]
[1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0]
[1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 1]
[1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0]
[1 1 0 1 1 0 0 1 0 1 1 1 0 1 0 1]
[1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
[1 1 0 0 1 1 0 1 0 1 1 1 0 1 0 1]
[1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0]
[1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1]
[1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0]


In [78]:
m = []
while len(m) < 8:
    m.append('fuck')

In [3]:
m = np.array([1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1], dtype='bool')

In [8]:
m = np.packbits(m)

In [12]:
int(m[0])

221

In [19]:
num = 0b11010010
composit = 0

for i in range(8):
#     print(1 & num  >> i)
    print(bin((1 & num  >> i) >> 2*i))

0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0


In [8]:
bin(composit)

'0b1001000001000000001000'